In [1]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import random


import keras
from matplotlib import pyplot as plt
%matplotlib inline
from keras import backend as K
from keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D, Conv2DTranspose
from keras.models import Model
from keras.optimizers import RMSprop
from keras import callbacks

Using TensorFlow backend.


In [2]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [3]:
df = pd.read_json(os.path.join('..', 'input', 'sample_analysis.json'))
df.head()

,analysis,features,track
0,"{'meta': {'analyzer_version': '4.0.0', 'platfo...","{'danceability': 0.559, 'energy': 0.33, 'key':...","{'album': {'album_type': 'album', 'artists': [..."
1,"{'meta': {'analyzer_version': '4.0.0', 'platfo...","{'danceability': 0.874, 'energy': 0.6920000000...","{'album': {'album_type': 'album', 'artists': [..."
2,"{'meta': {'analyzer_version': '4.0.0', 'platfo...","{'danceability': 0.759, 'energy': 0.548, 'key'...","{'album': {'album_type': 'album', 'artists': [..."
3,"{'meta': {'analyzer_version': '4.0.0', 'platfo...","{'danceability': 0.861, 'energy': 0.603, 'key'...","{'album': {'album_type': 'album', 'artists': [..."
4,"{'meta': {'analyzer_version': '4.0.0', 'platfo...","{'danceability': 0.777, 'energy': 0.782, 'key'...","{'album': {'album_type': 'album', 'artists': [..."


In [4]:
def segment_to_vector(segment):
    return [segment['duration'], segment['loudness_start'], segment['loudness_max_time'], segment['loudness_max']]\
        + segment['pitches'] + segment['timbre']

arrs = [list(map(segment_to_vector, sample['segments'])) for sample in df['analysis']]
del df

In [5]:
# arrs = list(map(np.array, arrs))
dim2_length = len(arrs[0][0])
to_pad = 428 #max(map(len, arrs))

for i in tqdm(range(len(arrs))):
    if len(arrs[i]) >= to_pad:
        arrs[i] = arrs[i][:to_pad]
    else:
        arrs[i] += [np.zeros(dim2_length) for _ in range(to_pad - len(arrs[i]))]

100%|█████████████████████████████████████████████████████████████████████████████| 510/510 [00:00<00:00, 18939.26it/s]


In [6]:
arrs = np.array(arrs)
arrs = arrs.reshape([*list(arrs.shape), 1])
arrs.shape

(510, 428, 28, 1)

In [7]:
batch_size = 8
epochs = 50
input_image = Input(shape=arrs.shape[1:])

In [8]:
from datetime import datetime
dt = datetime.today() 
currentDate = ''.join([str(dt.year), str(dt.month), str(dt.day)])

bestModelName = ''.join([currentDate,'AutoEncoder','.h5'])

In [9]:
#encoder
#input = 428 x 600 x 3
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1')(input_image) #428 x 600 x 32
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv2')(pool1) #214 x 300 x 64
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv3')(pool2) #107 x 150 x 128 

#decoder
conv4 = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv4')(conv3) #107 x 150 x 128 
up1 = UpSampling2D((2,2))(conv4) # 14 x 14 x 128
conv5 = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv5')(up1) #214 x 300 x 64
up2 = UpSampling2D((2,2))(conv5) # 28 x 28 x 64
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same', name='conv6')(up2) #428 x 600 x 3

autoencoder = Model(input_image, decoded)

early_stopping = callbacks.EarlyStopping(monitor='val_loss',patience=3, verbose=True)

bestModelFilepath = os.path.join('.','models',bestModelName)
checkpoint = callbacks.ModelCheckpoint(bestModelFilepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

loss = keras.losses.mse
optimizer = keras.optimizers.Adam()
history = keras.callbacks.History()

autoencoder.compile(loss=loss, optimizer = optimizer)
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 428, 28, 1)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 428, 28, 32)       320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 214, 14, 32)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 214, 14, 64)       18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 107, 7, 64)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 107, 7, 128)       73856     
_________________________________________________________________
conv4 (Conv2D)               (None, 107, 7, 128)       147584    
__________

In [10]:
from sklearn.model_selection import train_test_split

random_state=23
test_size=0.2

X_train, X_test = train_test_split(arrs,
                                   test_size=test_size,
                                   random_state=random_state)

In [ ]:
autoencoder.fit(X_train, X_train, batch_size=batch_size,epochs=epochs, \
                                    callbacks=[early_stopping, checkpoint, history], verbose=True,validation_split=0.1)

Train on 367 samples, validate on 41 samples
Epoch 1/50
